Temporary notebook, to preprocess data so it is in correct format for input to various notebooks. This preprocessing will be done in the preceding step in the chain but haven't added the cleaned up notebooks for each step yet.

In [ ]:
import pandas as pd
import re
import string

from pathlib import Path


In [ ]:
# Read the raw BDD dataset, containing the model outputs
df = pd.read_csv('../data/gherkins/exp2_p1_g04_10-10-25_first_5_us.csv')

df.head()

In [ ]:
df.rename(columns={'user_story': 'us_text', 'created': 'timestamp_response_generated', 'assistant_response': 'ai_response'}, inplace=True)

df.drop(columns=['timestamp'], inplace=True)
df['app_id'] = df.us_id.str.split('_').str[0]
df['us_id'] = df['us_id'].str.split('_').str[1]
df['model'] = df['model'].str.replace('/', '-')

# Adding a column to mark the experiment number - to map to spreadsheet
df['experiment'] = "test"

In [ ]:
df.head()

In [ ]:
df_reordered = df.iloc[:, [8, 0 , 7, 1, 2 , 3, 4, 5, 6]]

In [ ]:
df_reordered.head()

In [ ]:
exp_dir = Path("../data/gherkins/sample_data/test")

In [ ]:
df_reordered.to_csv(exp_dir / "test_preprocess_input.csv", index=False)

In [ ]:
# # Get path to active chat log file for a given model
# def active_file_path(chat_log_dir, model):
#     file_path = chat_log_dir / model / "active.json"

#     return file_path

In [ ]:
# # Helper function to find path to the latest chat log for a specific model
# def find_latest_log_file(log_dir, model):
#     files = [
#         f for f in os.listdir(log_dir)
#         if f.endswith(".json")
#     ]

#     if files:
#         files.sort(reverse=True)
#         return os.path.join(log_dir, files[0])
    
#     return None

In [ ]:
# # Helper function to find path to the latest chat log for a specific model
# def find_latest_log_file(log_dir, model):
#     files = [
#         f for f in os.listdir(log_dir)
#         if f.endswith(".json")
#     ]

#     if files:
#         files.sort(reverse=True)
#         return os.path.join(log_dir, files[0])
    
#     return None

In [ ]:
# #  Helper function to load latest chat log from JSON and rebuild messages
# def load_conversation(filepath):
#     with open(filepath, "r", encoding="utf-8") as f:
#         conversation_log = json.load(f)

#     messages = []

#     # Add system prompt if it exists
#     if conversation_log.get("system_prompt"):
#         messages.append({
#             "role": "system", 
#             "content": conversation_log["system_prompt"]
#         })

#     for turn in conversation_log["conversation"]:
#         messages.append({
#             "role": turn["role"], 
#             "content": turn["content"]
#         })

#     completed_stories = [
#         turn["us_id"] for turn in conversation_log["conversation"] 
#         if turn.get("role") == "user" and "us_id" in turn
#     ]

#     return messages, conversation_log, completed_stories

In [ ]:
async def chat_with_model(model,user_stories, or_token, log_dir, system_prompt=None, reminder=None, temperature=0.8, resume=True):
    model_log_dir = log_dir / model
    os.makedirs(model_log_dir, exist_ok=True)

    headers = {
        "Authorization": f"Bearer {or_token}",
        "Content-Type": "application/json",
    }

    # Find latest conversation log for this model
    # log_file_path = find_latest_conversation(log_dir, model)

    # if log_file_path and resume:
        # print(f"Resuming from {os.path.basename(log_file_path)}")
        # messages, conversation_log, completed_stories = load_conversation(log_file_path)

    # else:
        print(f"Starting new chat for {model}")

        # Initialise new conversation log, this will be written to file after each turn
        conversation_log = {
            "model": model,
            "timestamp": datetime.now().isoformat(),
            "system_prompt": system_prompt or "",
            "resumed_from": os.path.relpath(log_file_path) if (log_file_path and resume) else None,
            "conversation": [],
        }

        messages = []

        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})

        completed_stories = []

    async with httpx.AsyncClient() as client:
        for us_id, story in user_stories.items():
            if us_id in completed_stories:
                print(f"Skipping completed story: {us_id}")
                continue

            if reminder:
                messages.append({
                    "role": "user", 
                    "content": reminder
                })

                conversation_log["conversation"].append({
                    "role": "user", 
                    "content": reminder
                })

            messages.append({
                "role": "user", 
                "content": story
            })

            conversation_log["conversation"].append({
                "role": "user", 
                "content": story, 
                "us_id": us_id
            })

            # Send request
            try:
                response = await client.post(
                    url="https://openrouter.ai/api/v1/chat/completions",
                    headers=headers,
                    json={
                        "model": model,
                        "messages": messages,
                        "temperature": temperature,
                        "provider": {"data_collection": "deny"},
                    },
                )
                response.raise_for_status()

                data = response.json()
                reply = data["choices"][0]["message"]["content"]
                created = data.get("created", "")


            except Exception as e:
                print(f"Error processing {us_id}: {e}")
                continue  # Skip but keep loop running

            # Add assistant reply
            messages.append({"role": "assistant", "content": reply})
            conversation_log["conversation"].append(
                {"role": "assistant", "content": reply}
            )

            # Save checkpoint after each turn
            save_conversation(log_dir, model, conversation_log)

            # Respect rate limits
            await asyncio.sleep(1)

    print(
        f"Chat completed for {model}. "
        f"Total turns: {len(conversation_log['conversation'])}"
    )
    
    return conversation_log


In [ ]:
# Function to run multi-turn chat for one model, resuming if a saved log exists
async def chat_with_model(model, user_stories, or_token):
    headers = {
        "Authorization": f"Bearer {or_token}",
        "Content-Type": "application/json"
    }

    log_file = find_latest_conversation(model)

    if log_file:
        print(f"Resuming from {log_file}")

        messages, conversation_log, completed_stories = load_conversation(log_file)
    else:
        print(f"Starting new chat for {model}")

        messages = [
            {
                "role": "system", "content": (
                    prompt
            )}
        ]
        
        conversation_log = {
            "model": model,
            "timestamp": datetime.now().isoformat(),
            "conversation": []
        }

        completed_stories = []

    async with httpx.AsyncClient() as client:
        for us_id, story in user_stories.items():
            if us_id in completed_stories:
                print(f"Skipping completed story: {us_id}")

                continue

            if reminder:
                messages.append({"role": "user", "content": reminder})

            messages.append({"role": "user", "content": story})

            response = await client.post(
                url="https://openrouter.ai/api/v1/chat/completions",
                headers=headers,
                json={
                    "model": model,
                    "messages": messages,
                    "temperature": 0.8,
                    "provider": {
                        "data_collection": "deny"
                        }
                }
            )

            data = response.json()
            reply = data["choices"][0]["message"]["content"]
            messages.append({"role": "assistant", "content": reply})

            conversation_log["conversation"].append({
                "us_id": us_id,
                "user_story": story,
                "assistant_response": reply,
                "raw_response": data
            })

            save_conversation(conversation_log, model)
            await asyncio.sleep(1)  # short pause for rate limits

    return conversation_log

In [ ]:
# Manual prf
# Calculate precision, recall, and F1-score
prf = []

for (metric, model), group in predicted_matches.groupby(["metric", "model"]):
    for us in group["true_us"].unique():

        true_pos = ((group["predicted_us"] == us) & (group["true_us"] == us)).sum()
        false_pos = ((group["predicted_us"] == us) & (group["true_us"] != us)).sum()

        false_neg = ((group["predicted_us"] != us) & (group["true_us"] == us)).sum()

        precision = true_pos / (true_pos + false_pos) if (true_pos + false_pos) > 0 else 0
        recall = true_pos / (true_pos + false_neg) if (true_pos + false_neg) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        prf.append({
            "metric": metric,
            "model": model,
            "us_id": us,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        })


In [ ]:
# Function to generate and save embeddings (us or scenario) to pickle file
def generate_and_save_embeddings(ids, texts, embedding_model, filename):
    embeddings = embedding_model.encode(texts, show_progress_bar=True)

    embeddings_df = pd.DataFrame({
        "id": ids,
        "embedding": embeddings.tolist()
    })

    os.makedirs(experiment_path / 'embeddings', exist_ok=True)
    embeddings_df.to_pickle(experiment_path / 'embeddings' / filename)


In [ ]:
# # Generate and save US embeddings using each embedding model
# for embed_model_name, embed_model in embed_model_dict.items():
#     generate_and_save_embeddings(
#         us_ids,
#         us_texts,
#         embed_model,
#         f'us_embeddings_{embed_model_name}.pkl'
#     )

In [ ]:
for embed_model_name, embed_model in embed_model_dict.items():
    for llm, group in df.groupby("model"):
        scenario_ids = group["scenario_id"].tolist()
        scenario_texts = group["scenario_text"].tolist()

        generate_and_save_embeddings(
            scenario_ids,
            scenario_texts,
            embed_model,
            f'{llm}_scenario_embeddings_{embed_model_name}.pkl'
        )

In [ ]:
results = []

for embed_model_name in embed_model_dict.keys():
    # Load US embeddings from pickle
    us_embeddings_df = pd.read_pickle(experiment_path / f"embeddings/us_embeddings_{embed_model_name}.pkl")

    us_ids = us_embeddings_df["id"].tolist()
    us_embeddings = np.stack(us_embeddings_df["embedding"].values) # Convert to numpy array for cosine similarity computation

    for llm in llms:
        # Load scenario embeddings from pickle
        scenario_embeddings_df = pd.read_pickle(embeddings_path / f"{llm}_scenario_embeddings_{embed_model_name}.pkl")

        scenario_ids = scenario_embeddings_df["id"].tolist()
        scenario_embeddings = np.stack(scenario_embeddings_df["embedding"].values)

        # Compute cosine similarity
        cosine_sim_matrix = cosine_similarity(us_embeddings, scenario_embeddings)

        # NOTE: we don't preprocess texts for embeddings so preprocessed fields won't be included here
        current_results = [{
            "model": llm, 
            "us_id": us_ids[i],
            "scenario_id": scenario_ids[j],
            "metric": f"{embed_model_name}_embedding_cosine-sim",
            "similarity_score": cosine_sim_matrix[i, j]
        }
        for i in range(len(us_ids)) 
        for j in range(len(scenario_ids))
        ]

        results.extend(current_results)


In [ ]:
# Add us_text and scenario_text to sim_df
sim_df = (
    sim_df
    .merge(df[['scenario_id', 'scenario_text']], on="scenario_id", how="left")
    .merge(us_df[['us_id', 'us_text']], on="us_id", how="left")
)